In [1]:
from census import Census
from us import states
import pandas as pd
import geopandas as gpd
import json
from pandas import DataFrame
import censusdata
import functools as ft

In [2]:
f = open('config/keys.json')
data = json.load(f)
acs_api_key = data['acs']['key']
c = Census(acs_api_key)

In [26]:
# geo includes tract and blockgroup
def get_ACS5_data(tablename, fields, geo, year):
    if fields == "*":
        if year > 2019:
            YEAR = 2019
        else:
            YEAR = year
        fields = [key for key, value in censusdata.censustable('acs5', YEAR, tablename).items()]
    
    if tablename[0] == 'S':
        func_nm = 'acs5st'
    elif tablename[0] == 'D':
        func_nm = 'acs5dp'
    else:
        func_nm = 'acs5'
    
    ctycode = "039"
    al = "*"
  
    data = DataFrame(eval(f'c.{func_nm}.state_county_{geo}(fields = fields, state_fips = states.OR.fips, county_fips = ctycode, {geo} = al,year = year)'))
    
    if geo == "tract":
        data["GEOID"] = data["state"] + data["county"] + data["tract"]
        data = data.drop(columns = ["state", "county", "tract"])
        
    else:
        data["GEOID"] = data["state"] + data["county"] + data["tract"] + data["block group"]
        data = data.drop(columns = ["state", "county", "tract", "block group"])
        
    return(data)

In [34]:
def get_ACS5_years(tablename, fields, geo, years):
    data = pd.DataFrame()
    for year in years:
        ndata = get_ACS5_data(tablename = tablename, 
                             fields = fields, 
                             geo = geo, 
                             year = year)
        ndata['Year'] = year
        data = data.append(ndata)
        print(tablename + ' ' + str(year))
    return(data)

In [35]:
lane_pop_tract = get_ACS5_years(tablename = 'B01001', fields = '*', geo = 'tract', years = range(2012, 2021))

B01001 2012
B01001 2013
B01001 2014
B01001 2015
B01001 2016
B01001 2017
B01001 2018
B01001 2019
B01001 2020


In [37]:
lane_race_tract = get_ACS5_years(tablename = 'B03002', 
                                 fields = ['B03002_001E', 'B03002_003E'], 
                                 geo = 'tract', 
                                 years = range(2012, 2021))

B03002 2012
B03002 2013
B03002 2014
B03002 2015
B03002 2016
B03002 2017
B03002 2018
B03002 2019
B03002 2020


In [41]:
lane_employment_tract = get_ACS5_years(tablename = 'B23025', 
                                       fields = ['B23025_002E', 'B23025_004E'], 
                                       geo = 'tract', 
                                       years = range(2012, 2021))

B23025 2012
B23025 2013
B23025 2014
B23025 2015
B23025 2016
B23025 2017
B23025 2018
B23025 2019
B23025 2020


In [42]:
lane_income_tract = get_ACS5_years(tablename = 'S1901', 
                                       fields = ['S1901_C01_001E', 'S1901_C01_012E', 'S1901_C01_013E'], 
                                       geo = 'tract', 
                                       years = range(2012, 2021))

S1901 2012
S1901 2013
S1901 2014
S1901 2015
S1901 2016
S1901 2017
S1901 2018
S1901 2019
S1901 2020


In [51]:
lane_commute_tract = get_ACS5_years(tablename = 'B08301, B08303', 
                                       fields = ['B08301_001E', 'B08301_018E', 'B08303_001E', 'B08303_002E', 'B08303_003E',
                                                'B08303_004E', 'B08303_005E', 'B08303_006E', 'B08303_007E'], 
                                       geo = 'tract', 
                                       years = range(2012, 2021))

B08301, B08303 2012
B08301, B08303 2013
B08301, B08303 2014
B08301, B08303 2015
B08301, B08303 2016
B08301, B08303 2017
B08301, B08303 2018
B08301, B08303 2019
B08301, B08303 2020


In [40]:
#censusdata.printtable(censusdata.censustable('acs5', 2019, 'S1901'))

In [52]:
#censusdata.censustable('acs5', 2019, 'B08303')

In [54]:
dfs = [lane_pop_tract, lane_race_tract, lane_employment_tract, lane_income_tract, lane_commute_tract]

In [59]:
df_final = ft.reduce(lambda left, right: pd.merge(left, right, on=['GEOID', 'Year']), dfs)

In [60]:
print(df_final.head(2))

   B01001_001E  B01001_002E  B01001_003E  B01001_004E  B01001_005E  \
0       5065.0       2447.0        192.0        296.0        206.0   
1       4944.0       2477.0        148.0        105.0        158.0   

   B01001_006E  B01001_007E  B01001_008E  B01001_009E  B01001_010E  ...  \
0        112.0         68.0         49.0         21.0         59.0  ...   
1         48.0        113.0         17.0          0.0         47.0  ...   

   S1901_C01_013E  B08301_001E  B08301_018E  B08303_001E  B08303_002E  \
0         46805.0       1854.0         21.0       1656.0         87.0   
1         63775.0       2079.0          0.0       1906.0         37.0   

   B08303_003E  B08303_004E  B08303_005E  B08303_006E  B08303_007E  
0        303.0        263.0         93.0         90.0        124.0  
1         55.0        136.0        107.0        221.0        262.0  

[2 rows x 67 columns]
